# Chapter 4: Comprehensions and Generators

## Overview

This chapter explores Python's powerful features for working with sequences and iterators:

- **Comprehensions**: Concise syntax for creating lists, dictionaries, and sets
- **Generators**: Memory-efficient iteration using `yield`
- **Advanced patterns**: `yield from`, generator expressions, and `itertools`

---

## Item 27: Use Comprehensions Instead of map and filter

### Core Concept

List comprehensions provide a clearer, more Pythonic way to derive new lists from sequences compared to `map` and `filter` with `lambda` functions.

In [1]:
# Basic list comprehension example
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Traditional approach with loop
squares = []
for x in a:
    squares.append(x**2)
print("Loop approach:", squares)

# List comprehension - much clearer
squares = [x**2 
           for x in a]
print("Comprehension:", squares)

Loop approach: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Comprehension: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


### Comprehensions vs map()

In [2]:
# Using map (requires lambda, visually noisy)
alt = map(lambda x: x ** 2, a)
print("map result:", list(alt))

# List comprehension is clearer
squares = [x**2 
           for x in a]
print("Comprehension result:", squares)

map result: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Comprehension result: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


### Filtering with Comprehensions

In [3]:
# List comprehension with filtering
even_squares = [x**2 
                for x in a 
                if x % 2 == 0]
print("Even squares:", even_squares)


# Compare to map + filter (much harder to read)
alt = map(lambda x: x**2, filter(lambda x: x % 2 == 0, a))
print("map + filter:", list(alt))

# Verify they're equal
assert even_squares == list(map(lambda x: x**2, filter(lambda x: x % 2 == 0, a)))

Even squares: [4, 16, 36, 64, 100]
map + filter: [4, 16, 36, 64, 100]


### Dictionary and Set Comprehensions

In [ ]:
# Dictionary comprehension
even_squares_dict = {x: x**2 
                     for x in a 
                     if x % 2 == 0}
print("Dict comprehension:", even_squares_dict)

# Set comprehension
threes_cubed_set = {x**3 
                    for x in a 
                    if x % 3 == 0}
print("Set comprehension:", threes_cubed_set)

Dict comprehension: {2: 4, 4: 16, 6: 36, 8: 64, 10: 100}
Set comprehension: {216, 729, 27}


### Comparison: Comprehensions vs map/filter for Complex Operations

In [ ]:
# Comprehensions - clean and readable
even_squares_dict = {x: x**2 
                     for x in a 
                     if x % 2 == 0}
threes_cubed_set = {x**3 
                    for x in a 
                    if x % 3 == 0}

# map + filter equivalent (breaks across multiple lines, hard to read)
alt_dict = dict(map(lambda x: (x, x**2),
                   filter(lambda x: x % 2 == 0, a)))
alt_set = set(map(lambda x: x**3,
                 filter(lambda x: x % 3 == 0, a)))

print("Dict - comprehension:", even_squares_dict)
print("Dict - map/filter:   ", alt_dict)
print("\nSet - comprehension:", threes_cubed_set)
print("Set - map/filter:   ", alt_set)

Dict - comprehension: {2: 4, 4: 16, 6: 36, 8: 64, 10: 100}
Dict - map/filter:    {2: 4, 4: 16, 6: 36, 8: 64, 10: 100}

Set - comprehension: {216, 729, 27}
Set - map/filter:    {216, 729, 27}


### Enhanced Examples

In [ ]:
# Example 1: Processing strings
words = ['hello', 'world', 'python', 'comprehensions']
uppercase = [word.upper() 
             for word in words]
print("Uppercase:", uppercase)

# Example 2: Filtering by length
long_words = [word 
              for word in words 
              if len(word) > 5]
print("Long words:", long_words)

# Example 3: Creating dictionary from two lists
keys = ['a', 'b', 'c']
values = [1, 2, 3]
mapping = {k: v for k, v in zip(keys, values)}
print("Mapping:", mapping)

# Example 4: Set of unique lengths
lengths = {len(word) 
           for word in words}
print("Unique lengths:", lengths)

Uppercase: ['HELLO', 'WORLD', 'PYTHON', 'COMPREHENSIONS']
Long words: ['python', 'comprehensions']
Mapping: {'a': 1, 'b': 2, 'c': 3}
Unique lengths: {5, 6, 14}


### Things to Remember

✦ List comprehensions are clearer than `map` and `filter` built-in functions because they don't require `lambda` expressions

✦ List comprehensions allow you to easily skip items from the input list, a behavior that `map` doesn't support without help from `filter`

✦ Dictionaries and sets may also be created using comprehensions

---

## Item 28: Avoid More Than Two Control Subexpressions in Comprehensions

### Core Concept

Comprehensions support multiple levels of looping and conditions, but beyond two control subexpressions, readability suffers dramatically.

### Multiple Loops in Comprehensions

In [ ]:
# Flattening a matrix (reasonable)
matrix = [[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]
flat = [x 
        for row in matrix 
        for x in row]
print("Flattened:", flat)

# Squaring each cell (still readable)
squared = [[x**2 for x in row] for row in matrix]
print("Squared:\n", squared)

Flattened: [1, 2, 3, 4, 5, 6, 7, 8, 9]
Squared:
 [[1, 4, 9], [16, 25, 36], [49, 64, 81]]


### When Comprehensions Become Too Complex

In [8]:
# Three levels - too hard to read!
my_lists = [
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]]
]

# This is getting hard to follow
flat = [x for sublist1 in my_lists
          for sublist2 in sublist1
          for x in sublist2]
print("Three-level flatten:", flat)

# Better alternative: normal loops
flat_alternative = []
for sublist1 in my_lists:
    for sublist2 in sublist1:
        flat_alternative.extend(sublist2)
print("Loop alternative:", flat_alternative)

Three-level flatten: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Loop alternative: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


### Multiple Conditions

In [ ]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Multiple conditions at same level (implicit AND)
b = [x 
     for x in a 
     if x > 4 
     if x % 2 == 0]
print("Multiple if:", b)

# Equivalent using 'and'
c = [x 
     for x in a 
     if x > 4 and x % 2 == 0]
print("Using 'and':", c)

assert b == c

Multiple if: [6, 8, 10]
Using 'and': [6, 8, 10]


### Conditions at Different Loop Levels (Avoid This!)

In [10]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Very hard to read: conditions at multiple levels
filtered = [[x for x in row if x % 3 == 0]
            for row in matrix if sum(row) >= 10]
print("Complex filter:", filtered)

# This says: "Include rows that sum to >= 10, 
# and within those rows, include only numbers divisible by 3"

Complex filter: [[6], [9]]


### When to Use Helper Functions

In [ ]:
# Instead of complex comprehension,use helper function 
def get_filtered_values(matrix):
    """Extract values divisible by 3 from rows that sum >= 10"""
    result = []
    for row in matrix:
        if sum(row) >= 10:
            filtered_row = [x for x in row if x % 3 == 0]
            if filtered_row:
                result.append(filtered_row)
    return result

matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
result = get_filtered_values(matrix)
print("Using helper function:", result)

Using helper function: [[6], [9]]


### Enhanced Examples: Good vs Bad Complexity

In [ ]:
# ============================================================================
# PYTHON LIST COMPREHENSION COMPLEXITY GUIDELINES
# ============================================================================
# This code demonstrates best practices for writing readable list comprehensions
# Key principle: Limit to TWO control subexpressions maximum
# ============================================================================

# ✓ GOOD EXAMPLE: Two control subexpressions
# ----------------------------------------------------------------------------

# Define a 2D matrix (list of lists)
matrix = [[1, 2, 3],   # First row
          [4, 5, 6]]   # Second row

# List comprehension with acceptable complexity
flat_evens = [x                    # Expression: take the element x
              for row in matrix    # Control 1: iterate through each row
              for x in row         # Control 2: iterate through elements in row
              if x % 2 == 0]       # Filter: only keep even numbers (divisible by 2)

# What this does:
# 1. Iterates through each row: [1,2,3], then [4,5,6]
# 2. For each row, iterates through each element
# 3. Tests if element is even (x % 2 == 0)
# 4. Keeps only even numbers: 2, 4, 6
# Result: flattens 2D matrix and filters for evens

print("Good:", flat_evens)  # Output: [2, 4, 6]


# ✗ BAD EXAMPLE: Three+ control subexpressions
# ----------------------------------------------------------------------------
# Don't do this! Three nested comprehensions become unreadable

# This would create a 3D nested list structure:
# nested = [[[x**2 for x in range(3)]  # Innermost: squares [0, 1, 4]
#            for _ in range(2)]         # Middle: repeat 2 times
#           for _ in range(2)]          # Outermost: repeat 2 times

# Why this is bad:
# - Hard to understand at a glance
# - Difficult to debug
# - Violates readability principle
# - Mental overhead too high


# ✓ BETTER ALTERNATIVE: Explicit loops for complex cases
# ----------------------------------------------------------------------------

def create_nested_structure():
    """
    Creates a 3D nested structure using explicit loops.
    Structure: 2 x 2 x 3 (outermost x middle x innermost dimensions)
    
    Returns a list like:
    [[[0, 1, 4], [0, 1, 4]], [[0, 1, 4], [0, 1, 4]]]
    """
    
    result = []  # Will hold the final 3D structure
    
    # Outermost loop: creates 2 major groups
    for _ in range(2):
        inner = []  # Will hold one major group (2D structure)
        
        # Middle loop: creates 2 sub-groups within each major group
        for _ in range(2):
            # Innermost operation: create list of squares [0, 1, 4]
            # This is simple enough to keep as a comprehension
            innermost = [x**2 for x in range(3)]  # [0**2, 1**2, 2**2] = [0, 1, 4]
            inner.append(innermost)  # Add to sub-group
        
        result.append(inner)  # Add completed sub-group to result
    
    return result

# Call the function to create the structure
nested = create_nested_structure()

# Output: [[[0, 1, 4], [0, 1, 4]], [[0, 1, 4], [0, 1, 4]]]
print("Better approach:", nested)


# ============================================================================
# KEY TAKEAWAYS
# ============================================================================
# 1. List comprehensions with 2 control subexpressions: ACCEPTABLE
# 2. List comprehensions with 3+ control subexpressions: USE EXPLICIT LOOPS
# 3. Readability > Conciseness (Python's Zen: "Readability counts")
# 4. When in doubt, use a function with explicit loops and clear variable names
# ============================================================================

Good: [2, 4, 6]
Better approach: [[[0, 1, 4], [0, 1, 4]], [[0, 1, 4], [0, 1, 4]]]


### Things to Remember

✦ Comprehensions support multiple levels of loops and multiple conditions per loop level

✦ Comprehensions with more than two control subexpressions are very difficult to read and should be avoided

✦ Use helper functions or normal loops when complexity exceeds two subexpressions

---

## Item 29: Avoid Repeated Work in Comprehensions by Using Assignment Expressions

### Core Concept

The walrus operator (`:=`) allows you to assign values within comprehensions, avoiding redundant computations and improving both readability and performance.

### The Problem: Repeated Computation

In [13]:
# Setup: order fulfillment system
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count // size

# Traditional loop approach
result = {}
for name in order:
    count = stock.get(name, 0)
    batches = get_batches(count, 8)
    if batches:
        result[name] = batches

print("Loop result:", result)

Loop result: {'screws': 4, 'wingnuts': 1}


### Problematic: Repeated Expression in Comprehension

In [14]:
# BAD: get_batches called twice (redundant computation)
found = {name: get_batches(stock.get(name, 0), 8)
         for name in order
         if get_batches(stock.get(name, 0), 8)}

print("Repeated computation:", found)

# This is error-prone! Easy to get out of sync:
has_bug = {name: get_batches(stock.get(name, 0), 4)  # Different size!
           for name in order
           if get_batches(stock.get(name, 0), 8)}

print("Expected:", found)
print("Bug version:", has_bug)
print("Results differ!" if found != has_bug else "Results match")

Repeated computation: {'screws': 4, 'wingnuts': 1}
Expected: {'screws': 4, 'wingnuts': 1}
Bug version: {'screws': 8, 'wingnuts': 2}
Results differ!


### Best Practice: Use in Condition Only

In [ ]:
# Recommended: Assignment expressions in the condition part
result = {name: batches 
          for name in order
          if (batches := get_batches(stock.get(name, 0), 8))}

print("Best practice:", result)

### Solution: Assignment Expressions (Walrus Operator)

In [ ]:
# ============================================================================
# PYTHON WALRUS OPERATOR (:=) - ASSIGNMENT EXPRESSIONS
# ============================================================================
# The walrus operator allows assignment within expressions
# Benefit: Compute once, use multiple times (avoid redundant calculations)
# ============================================================================

def get_batches(count, size):
    """
    Calculate how many batches are needed given a count and batch size.
    
    Args:
        count: Total number of items
        size: Number of items per batch
        
    Returns:
        Number of batches needed (0 if count is 0)
    """
    return 0 if count == 0 else (count // size) + (1 if count % size else 0)


# ============================================================================
# EXAMPLE 1: Using walrus operator in dictionary comprehension
# ============================================================================

# GOOD: Compute once, use multiple times
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips']

# Using walrus operator to avoid calling get_batches twice
found = {name: batches 
         for name in order
         if (batches := get_batches(stock.get(name, 0), 8))}

print("Assignment expression:", found)

# The := operator:
# 1. Evaluates get_batches(stock.get(name, 0), 8)
# 2. Assigns result to 'batches'
# 3. Returns the value for the if condition
# 4. 'batches' is available in the value expression


# ============================================================================
# COMPARISON: Without walrus operator (BAD - computes twice)
# ============================================================================

# BAD: Calling get_batches twice (inefficient)
found_without_walrus = {
    name: get_batches(stock.get(name, 0), 8)  # First call
    for name in order
    if get_batches(stock.get(name, 0), 8)     # Second call (duplicate!)
}

print("\nWithout walrus (inefficient):", found_without_walrus)


# ============================================================================
# EXAMPLE 2: Walrus operator in list comprehension
# ============================================================================

# Calculate batch counts for all items in stock
batch_info = [(name, batches) 
              for name in stock 
              if (batches := get_batches(stock[name], 8)) > 0]

print("\nBatch info:", batch_info)


# ============================================================================
# EXAMPLE 3: Walrus operator in while loop
# ============================================================================

print("\nReading user input (type 'quit' to stop):")

# Simulate user input for demonstration
simulated_inputs = ['hello', 'world', 'python', 'quit']
input_index = 0

def mock_input(prompt):
    """Mock input function for demonstration"""
    global input_index
    if input_index < len(simulated_inputs):
        value = simulated_inputs[input_index]
        input_index += 1
        print(f"{prompt}{value}")
        return value
    return 'quit'

# Using walrus operator in while condition
while (user_input := mock_input("Enter text: ")) != 'quit':
    print(f"  You entered: {user_input}")


# ============================================================================
# EXAMPLE 4: Walrus operator with filtering
# ============================================================================

# Filter and transform in one pass
numbers = [1, 5, 12, 3, 18, 7, 20, 15]

# Get doubled values for numbers > 10
doubled_large = [doubled for n in numbers 
                 if (doubled := n * 2) > 20]

print("\nDoubled values > 20:", doubled_large)


# ============================================================================
# DETAILED BREAKDOWN OF THE MAIN EXAMPLE
# ============================================================================

print("\n" + "="*70)
print("DETAILED BREAKDOWN")
print("="*70)

# Step-by-step explanation of what happens:
for name in order:
    stock_count = stock.get(name, 0)
    batches = get_batches(stock_count, 8)
    print(f"\nItem: {name}")
    print(f"  Stock count: {stock_count}")
    print(f"  Batches needed: {batches}")
    print(f"  Include in result? {batches > 0}")


# ============================================================================
# KEY BENEFITS OF WALRUS OPERATOR
# ============================================================================
print("\n" + "="*70)
print("KEY BENEFITS")
print("="*70)
print("1. Avoid redundant calculations (compute once, use multiple times)")
print("2. Cleaner code (no need for intermediate variables)")
print("3. Better performance (single function call instead of multiple)")
print("4. More expressive (assignment + evaluation in one expression)")

Assignment expression: {'screws': 4, 'wingnuts': 1}


In [24]:
# ============================================================================
# PYTHON WALRUS OPERATOR (:=) - SIMPLE ARITHMETIC EXAMPLES
# ============================================================================
# The walrus operator allows assignment within expressions
# Let's see it with basic math operations
# ============================================================================

# ============================================================================
# EXAMPLE 1: Simple addition - filtering results
# ============================================================================

print("EXAMPLE 1: Addition and filtering")
print("-" * 50)

numbers = [5, 10, 15, 20, 25, 30]

# WITHOUT walrus operator (BAD - calculates twice)
# We want numbers where (number + 10) is greater than 25
result_without = [n + 10 
                  for n in numbers 
                  if n + 10 > 25]  # Calculates n + 10 twice!

print("Without walrus:", result_without)

# WITH walrus operator (GOOD - calculates once)
result_with = [added 
               for n in numbers 
               if (added := n + 10) > 25]  # Calculate once, use twice!

print("With walrus:   ", result_with)
print()


# ============================================================================
# EXAMPLE 2: Subtraction - checking if result is positive
# ============================================================================

print("EXAMPLE 2: Subtraction")
print("-" * 50)

prices = [100, 75, 50, 30, 20]
discount = 40

# Get final prices only if they're still positive after discount
final_prices = [discounted 
                for price in prices 
                if (discounted := price - discount) > 0]

print(f"Original prices: {prices}")
print(f"Discount: ${discount}")
print(f"Final prices (positive only): {final_prices}")
print()


# ============================================================================
# EXAMPLE 3: Multiplication - filtering large results
# ============================================================================

print("EXAMPLE 3: Multiplication")
print("-" * 50)

quantities = [3, 5, 8, 12, 15]
price_per_item = 7

# Get total costs only if they exceed $50
expensive_orders = [total 
                    for qty in quantities 
                    if (total := qty * price_per_item) > 50]

print(f"Quantities: {quantities}")
print(f"Price per item: ${price_per_item}")
print(f"Orders over $50: {expensive_orders}")
print()


# ============================================================================
# EXAMPLE 4: Division - avoiding repeated calculations
# ============================================================================

print("EXAMPLE 4: Division")
print("-" * 50)

total_items = [100, 200, 300, 400, 500]
num_boxes = 30

# Calculate items per box, only include if there are at least 10 per box
items_per_box = [per_box 
                 for total in total_items 
                 if (per_box := total // num_boxes) >= 10]

print(f"Total items: {total_items}")
print(f"Number of boxes: {num_boxes}")
print(f"Items per box (≥10): {items_per_box}")
print()


# ============================================================================
# EXAMPLE 5: Combined operations - realistic scenario
# ============================================================================

print("EXAMPLE 5: Combined operations (profit calculation)")
print("-" * 50)

# Calculate profit: (selling_price - cost) * quantity
sales = [
    {'item': 'A', 'cost': 10, 'price': 15, 'qty': 5},
    {'item': 'B', 'cost': 20, 'price': 25, 'qty': 3},
    {'item': 'C', 'cost': 8, 'price': 12, 'qty': 10},
    {'item': 'D', 'cost': 50, 'price': 55, 'qty': 2},
]

# Get items where profit is at least $20
profitable = [(sale['item'], profit)
              for sale in sales
              if (profit := (sale['price'] - sale['cost']) * sale['qty']) >= 20]

print("Profitable items (profit ≥ $20):")
for item, profit in profitable:
    print(f"  Item {item}: ${profit}")
print()


# ============================================================================
# EXAMPLE 6: Step-by-step comparison
# ============================================================================

print("EXAMPLE 6: Step-by-step comparison")
print("-" * 50)

numbers = [10, 20, 30, 40]

print("\nWITHOUT walrus operator:")
print("Code: [n * 2 for n in numbers if n * 2 > 50]")
for n in numbers:
    doubled = n * 2
    print(f"  n={n}, n*2={doubled}, include={doubled > 50}")
result = [n * 2 for n in numbers if n * 2 > 50]
print(f"Result: {result}")
print("Problem: n * 2 calculated TWICE for each number!")

print("\nWITH walrus operator:")
print("Code: [doubled for n in numbers if (doubled := n * 2) > 50]")
for n in numbers:
    doubled = n * 2
    print(f"  n={n}, doubled={doubled}, include={doubled > 50}")
result = [doubled for n in numbers if (doubled := n * 2) > 50]
print(f"Result: {result}")
print("Benefit: n * 2 calculated ONCE for each number!")
print()


# ============================================================================
# KEY TAKEAWAY
# ============================================================================
print("=" * 70)
print("KEY TAKEAWAY")
print("=" * 70)
print("The walrus operator := lets you:")
print("  1. Perform calculation ONCE: (result := x + y)")
print("  2. Check the result: if result > 10")
print("  3. Use the result: return result")
print("\nAll in a single expression - no repeated calculations!")

EXAMPLE 1: Addition and filtering
--------------------------------------------------
Without walrus: [30, 35, 40]
With walrus:    [30, 35, 40]

EXAMPLE 2: Subtraction
--------------------------------------------------
Original prices: [100, 75, 50, 30, 20]
Discount: $40
Final prices (positive only): [60, 35, 10]

EXAMPLE 3: Multiplication
--------------------------------------------------
Quantities: [3, 5, 8, 12, 15]
Price per item: $7
Orders over $50: [56, 84, 105]

EXAMPLE 4: Division
--------------------------------------------------
Total items: [100, 200, 300, 400, 500]
Number of boxes: 30
Items per box (≥10): [10, 13, 16]

EXAMPLE 5: Combined operations (profit calculation)
--------------------------------------------------
Profitable items (profit ≥ $20):
  Item A: $25
  Item C: $40

EXAMPLE 6: Step-by-step comparison
--------------------------------------------------

WITHOUT walrus operator:
Code: [n * 2 for n in numbers if n * 2 > 50]
  n=10, n*2=20, include=False
  n=20, n*

In [ ]:
# ============================================================================
# EXAMPLE 2: Subtraction - checking if result is positive
# ============================================================================

print("EXAMPLE 2: Subtraction")
print("-" * 50)

prices = [100, 75, 50, 30, 20]
discount = 40

# Get final prices only if they're still positive after discount
# 
# WHAT THE WALRUS OPERATOR := DOES HERE:
# 
# This list comprehension is equivalent to the following loop:
#
# final_prices = []
# for price in prices:
#     # The := operator calculates AND assigns in one step
#     discounted := price - discount
#     │              │
#     │              └─> Calculate: price - discount
#     └────────────────> Store result in variable 'discounted'
#     
#     # Then immediately use 'discounted' in the condition
#     if discounted > 0:
#         # If true, add 'discounted' to the list
#         final_prices.append(discounted)
#
# KEY BENEFIT: We calculate (price - discount) ONCE and use it TWICE:
#   1. Store it in 'discounted'
#   2. Check if discounted > 0
#   3. Add 'discounted' to the list (not recalculating)
#
# WITHOUT := we would need to calculate (price - discount) twice:
#   [price - discount for price in prices if price - discount > 0]
#                ↑                                    ↑
#            calculated                          calculated again!
#
final_prices = [discounted 
                for price in prices 
                if (discounted := price - discount) > 0]

print(f"Original prices: {prices}")
print(f"Discount: ${discount}")
print(f"Final prices (positive only): {final_prices}")
print()

# EXECUTION TRACE:
# ----------------
# price=100: discounted=100-40=60  → 60>0? YES → include 60
# price=75:  discounted=75-40=35   → 35>0? YES → include 35
# price=50:  discounted=50-40=10   → 10>0? YES → include 10
# price=30:  discounted=30-40=-10  → -10>0? NO → exclude
# price=20:  discounted=20-40=-20  → -20>0? NO → exclude
#
# Result: [60, 35, 10]

### Placement Rules for Assignment Expressions

In [16]:
# ✗ WRONG: Assignment in value expression without condition
# This causes NameError!
try:
    result = {name: (tenth := count // 10)
              for name, count in stock.items() if tenth > 0}
except NameError as e:
    print(f"Error: {e}")

# ✓ CORRECT: Assignment in condition
result = {name: tenth for name, count in stock.items()
          if (tenth := count // 10) > 0}
print("Correct placement:", result)

Error: name 'tenth' is not defined
Correct placement: {'nails': 12, 'screws': 3, 'washers': 2}


### Variable Leakage Behavior

In [ ]:
# Assignment expressions WITHOUT condition leak the variable
half = [(last := count // 2) for count in stock.values()]
print(f'Last item of {half} is {last}')

# This is similar to regular for loops
for count in stock.values():
    pass
print(f'Last item is {count}')

# But loop variables in comprehensions DON'T leak
half = [count // 2 
        for count in stock.values()]
print(half)  # Works
try:
    print(count)  # This will fail
except NameError:
    print("'count' is not defined outside comprehension")

Last item of [62, 17, 4, 12] is 12
Last item is 24
[62, 17, 4, 12]
24


### Generator Expressions with Assignment Expressions

In [19]:
# Works the same way with generator expressions
found = ((name, batches) for name in order
         if (batches := get_batches(stock.get(name, 0), 8)))

print(next(found))
print(next(found))

('screws', 4)
('wingnuts', 1)


### Enhanced Examples

In [ ]:
# ============================================================================
# WALRUS OPERATOR (:=) EXAMPLES - Advanced Use Cases
# ============================================================================
# The walrus operator allows assignment within expressions
# This is useful when you need to use a calculated value multiple times
# ============================================================================

# ============================================================================
# Example 1: Processing with validation
# ============================================================================

def validate_and_transform(value):
    """
    Expensive validation and transformation function
    
    This simulates a function that:
    - Takes time to compute (expensive operation)
    - Returns None for invalid inputs
    - Returns transformed value for valid inputs
    
    Parameters:
    -----------
    value : int
        Input number to validate and transform
        
    Returns:
    --------
    int or None
        If even: returns value squared (value ** 2)
        If odd: returns None (invalid)
    """
    if value % 2 == 0:
        return value ** 2
    return None

numbers = [1, 2, 3, 4, 5, 6]

# WITHOUT walrus operator (inefficient - calls function twice):
# results = [validate_and_transform(x) for x in numbers 
#            if validate_and_transform(x) is not None]
#            └─ called here ─┘      └─ called again here ─┘
#
# WITH walrus operator (efficient - calls function once):
# 
# Expanded loop equivalent:
# results = []
# for x in numbers:
#     # The := operator assigns AND returns the value in one step
#     transformed := validate_and_transform(x)
#     │              │
#     │              └─> Call the function: validate_and_transform(x)
#     └────────────────> Store result in variable 'transformed'
#     
#     # Then use 'transformed' in the condition check
#     if transformed is not None:
#         # If not None, add 'transformed' to results
#         results.append(transformed)
#
# EXECUTION TRACE:
# x=1: transformed=None    → None? YES → exclude
# x=2: transformed=4       → None? NO  → include 4
# x=3: transformed=None    → None? YES → exclude
# x=4: transformed=16      → None? NO  → include 16
# x=5: transformed=None    → None? YES → exclude
# x=6: transformed=36      → None? NO  → include 36
#
results = [transformed for x in numbers 
           if (transformed := validate_and_transform(x)) is not None]
print("Validated:", results)  # Output: [4, 16, 36]


# ============================================================================
# Example 2: String processing
# ============================================================================

words = ['hello', 'world', '', 'python', 'programming']

# Get uppercase versions of words, but only if they're longer than 5 characters
#
# WITHOUT walrus operator (calls .upper() twice):
# uppercase_long = [word.upper() for word in words 
#                   if word.upper() and len(word.upper()) > 5]
#                      └─ here ─┘          └─ again here ─┘
#
# WITH walrus operator (calls .upper() once):
#
# Expanded loop equivalent:
# uppercase_long = []
# for word in words:
#     # The := operator converts to uppercase AND stores it
#     upper := word.upper()
#     │        │
#     │        └─> Convert word to uppercase
#     └──────────> Store in variable 'upper'
#     
#     # Use 'upper' in compound condition
#     if upper and len(upper) > 5:
#         │        │
#         │        └─> Check if length > 5
#         └──────────> Check if not empty (truthy check)
#         
#         # If both conditions true, add 'upper' to list
#         uppercase_long.append(upper)
#
# EXECUTION TRACE:
# word='hello':       upper='HELLO'       → truthy? YES, len=5>5? NO  → exclude
# word='world':       upper='WORLD'       → truthy? YES, len=5>5? NO  → exclude
# word='':            upper=''            → truthy? NO               → exclude
# word='python':      upper='PYTHON'      → truthy? YES, len=6>5? YES → include 'PYTHON'
# word='programming': upper='PROGRAMMING' → truthy? YES, len=11>5? YES → include 'PROGRAMMING'
#
uppercase_long = [upper 
                  for word in words 
                  if (upper := word.upper()) and len(upper) > 5]
print("Long uppercase:", uppercase_long)  # Output: ['PYTHON', 'PROGRAMMING']


# ============================================================================
# Example 3: Mathematical computation
# ============================================================================

import math
values = [1, 4, 9, 16, 25]

# Create tuples of (original, square_root, log10) but only for values
# whose square root is greater than 2
#
# WITHOUT walrus operator (calls sqrt twice):
# roots_and_logs = [(val, math.sqrt(val), math.log10(val)) 
#                   for val in values 
#                   if math.sqrt(val) > 2]
#                      └─ here ─┘  └─ again ─┘
#
# WITH walrus operator (calls sqrt once):
#
# Expanded loop equivalent:
# roots_and_logs = []
# for val in values:
#     # The := operator calculates square root AND stores it
#     root := math.sqrt(val)
#     │       │
#     │       └─> Calculate: square root of val
#     └─────────> Store result in variable 'root'
#     
#     # Check condition using the stored 'root'
#     if root > 2:
#         # Create tuple using 'root' (don't recalculate)
#         # Also calculate log10 for this value
#         tuple_result = (val, root, math.log10(val))
#         roots_and_logs.append(tuple_result)
#
# EXECUTION TRACE:
# val=1:  root=√1=1.0   → 1.0>2? NO  → exclude
# val=4:  root=√4=2.0   → 2.0>2? NO  → exclude
# val=9:  root=√9=3.0   → 3.0>2? YES → include (9, 3.0, log10(9)=0.954)
# val=16: root=√16=4.0  → 4.0>2? YES → include (16, 4.0, log10(16)=1.204)
# val=25: root=√25=5.0  → 5.0>2? YES → include (25, 5.0, log10(25)=1.398)
#
roots_and_logs = [(val, root, math.log10(val)) 
                  for val in values 
                  if (root := math.sqrt(val)) > 2]
print("Roots and logs:", roots_and_logs)
# Output: [(9, 3.0, 0.954...), (16, 4.0, 1.204...), (25, 5.0, 1.398...)]


# ============================================================================
# KEY TAKEAWAYS
# ============================================================================
print("\n" + "=" * 70)
print("WALRUS OPERATOR BENEFITS:")
print("=" * 70)
print("""
1. EFFICIENCY: Calculate expensive operations only once
   - Function calls: validate_and_transform(x) called once, not twice
   - String operations: word.upper() called once, not twice
   - Math operations: math.sqrt(val) called once, not twice

2. READABILITY: Give meaningful names to intermediate values
   - 'transformed' is clearer than repeating the function call
   - 'upper' is more readable than word.upper() everywhere
   - 'root' better expresses intent than math.sqrt(val)

3. MAINTAINABILITY: Change the calculation in only one place
   - If the transformation logic changes, update it once
   - No risk of inconsistency from duplicate expressions

SYNTAX REMINDER:
----------------
if (variable := expression) condition:
    │         │              │
    │         │              └─> Use 'variable' in condition
    │         └────────────────> Calculate and assign
    └──────────────────────────> New variable name

The := operator:
- Assigns the value to the variable
- Returns the value so it can be used immediately
- Must be wrapped in parentheses when used in conditions
""")

Validated: [4, 16, 36]
Long uppercase: ['PYTHON', 'PROGRAMMING']
Roots and logs: [(9, 3.0, 0.9542425094393249), (16, 4.0, 1.2041199826559248), (25, 5.0, 1.3979400086720377)]


### Things to Remember

✦ Assignment expressions make it possible for comprehensions and generator expressions to reuse the value from one condition elsewhere in the same comprehension, which can improve readability and performance

✦ Although it's possible to use an assignment expression outside of a comprehension or generator expression's condition, you should avoid doing so

✦ Use assignment expressions in the condition part to avoid variable leakage

---

## Item 30: Consider Generators Instead of Returning Lists

### Core Concept

Generators provide a memory-efficient way to produce sequences of values without storing the entire result in memory. They're especially useful for large or infinite sequences.

### Traditional Approach: Returning Lists

In [ ]:
def index_words(text):
    """Find the index of each word in a string"""
    # Initialize empty list to store starting indices of each word
    result = []
    
    # Check if text is not empty (avoid edge case of empty string)
    if text:
        # First word always starts at index 0
        result.append(0)
    
    # Loop through each character with its index position
    for index, letter in enumerate(text):
        # Check if current character is a space (indicates word boundary)
        if letter == ' ':
            # Add the index of the next character (start of next word)
            result.append(index + 1)
    
    # Return list of all word starting positions
    return result


# Test string containing multiple words
address = 'Four score and seven years ago...'

# Call function to get list of indices where each word starts
result = index_words(address)

# Print first 10 word indices (slice in case there are many words)
print("Word indices:", result[:10])

Word indices: [0, 5, 11, 15, 21, 27]


### Problems with the List Approach

**Problem 1**: Code is dense and noisy
- Method calls (`result.append`) add bulk
- Separate lines for creating and returning the list

**Problem 2**: Memory usage
- All results must be stored before returning
- Can cause crashes with huge inputs

### Solution: Generator Function

In [22]:
def index_words_iter(text):
    """Generator version - much cleaner"""
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

# Generator doesn't run immediately
it = index_words_iter(address)
print("Iterator object:", it)

# Advance with next()
print("First index:", next(it))
print("Second index:", next(it))

# Or convert to list
result = list(index_words_iter(address))
print("All indices:", result[:10])

Iterator object: <generator object index_words_iter at 0x000001F6B500C970>
First index: 0
Second index: 5
All indices: [0, 5, 11, 15, 21, 27]


### Streaming Large Inputs

In [ ]:
# # Create a test file first
# #with open('/home/claude/address.txt', 'w') as f:
#    # f.write(address)

# def index_file(handle):
#     """Stream from file - bounded memory usage"""
#     offset = 0
#     for line in handle:
#         if line:
#             yield offset
#         for letter in line:
#             offset += 1
#             if letter == ' ':
#                 yield offset

# import itertools

# with open('/home/claude/address.txt', 'r') as f:
#     it = index_file(f)
#     results = itertools.islice(it, 0, 10)
#     print("From file:", list(results))

### How Generators Work

In [25]:
def demo_generator():
    print("Starting")
    yield 1
    print("After first yield")
    yield 2
    print("After second yield")
    yield 3
    print("Done")

# Generator doesn't execute until iterated
gen = demo_generator()
print("Generator created\n")

print("First next():", next(gen))
print("Second next():", next(gen))
print("Third next():", next(gen))

try:
    next(gen)
except StopIteration:
    print("\nGenerator exhausted")

Generator created

Starting
First next(): 1
After first yield
Second next(): 2
After second yield
Third next(): 3
Done

Generator exhausted


### Enhanced Examples

In [ ]:
# Example 1: Fibonacci generator
def fibonacci(n):
    """Generate first n Fibonacci numbers"""
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b

print("First 10 Fibonacci:", list(fibonacci(10)))

# Example 2: Infinite sequence
def count_forever(start=0):
    """Infinite counter"""
    while True:
        yield start
        start += 1

counter = count_forever(1)
print("First 5 counts:", [next(counter) for _ in range(5)])

# Example 3: Processing pipeline
def read_numbers(filename):
    """Read numbers from file"""
    with open(filename) as f:
        for line in f:
            yield int(line.strip())

def filter_even(numbers):
    """Filter even numbers"""
    for num in numbers:
        if num % 2 == 0:
            yield num

def square(numbers):
    """Square numbers"""
    for num in numbers:
        yield num ** 2

# Create test file
with open('/home/claude/numbers.txt', 'w') as f:
    f.write('\n'.join(map(str, range(1, 11))))

# Chain generators together (memory efficient!)
pipeline = square(filter_even(read_numbers('/home/claude/numbers.txt')))
print("Pipeline result:", list(pipeline))

### Gotcha: Stateful Iterators

In [ ]:
# Important: generators are stateful and can't be reused
gen = fibonacci(5)
print("First use:", list(gen))
print("Second use:", list(gen))  # Empty!

# Need to create new generator for reuse
gen = fibonacci(5)
print("New generator:", list(gen))

### Things to Remember

✦ Using generators can be clearer than the alternative of having a function return a list of accumulated results

✦ The iterator returned by a generator produces the set of values passed to `yield` expressions within the generator function's body

✦ Generators can produce a sequence of outputs for arbitrarily large inputs because their working memory doesn't include all inputs and outputs

✦ Generators are stateful and can't be reused

---

## Practice Exercises

### Exercise 1: List Comprehensions
Create a list comprehension that generates perfect squares from 1 to 100 that are divisible by 3.

In [26]:
# Your solution here
result = [x**2 
          for x in range(1, 11) 
          if x**2 % 3 == 0]
print(result)

[9, 36, 81]


### Exercise 2: Dictionary Comprehension
Create a dictionary mapping words to their lengths, only including words longer than 4 characters.

In [27]:
# Your solution here
words = ['cat', 'elephant', 'dog', 'giraffe', 'ant']
result = {word: len(word) 
          for word in words 
          if len(word) > 4}
print(result)

{'elephant': 8, 'giraffe': 7}


### Exercise 3: Generator Function
Write a generator that yields prime numbers up to n.

In [28]:
# Your solution here
def primes_up_to(n):
    for num in range(2, n + 1):
        is_prime = True
        for divisor in range(2, int(num ** 0.5) + 1):
            if num % divisor == 0:
                is_prime = False
                break
        if is_prime:
            yield num

print("Primes up to 30:", list(primes_up_to(30)))

Primes up to 30: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


## Summary

### Key Takeaways

1. **Comprehensions are Pythonic**: Use list/dict/set comprehensions instead of `map`/`filter` with lambdas

2. **Keep comprehensions simple**: Limit to 2 control subexpressions maximum

3. **Use walrus operator wisely**: Assignment expressions (`:=`) avoid repeated computation in comprehensions

4. **Generators save memory**: Use generators for large sequences or streaming data

5. **Know the tradeoffs**: Generators are single-use; lists can be reused

### Pattern Summary

```python
# List comprehension with filter
[expression for item in iterable if condition]

# Dictionary comprehension
{key_expr: value_expr for item in iterable if condition}

# Set comprehension
{expression for item in iterable if condition}

# Generator expression
(expression for item in iterable if condition)

# Assignment expression in comprehension
[value for item in iterable if (value := transform(item))]

# Generator function
def generator():
    for item in iterable:
        yield process(item)
```

---

## Next Steps

In the next items, we'll explore:
- Defensive iteration patterns (Item 31)
- Generator expressions for large comprehensions (Item 32)
- Composing generators with `yield from` (Item 33)
- Advanced generator techniques
- The `itertools` module

Continue practicing these patterns - they're fundamental to writing Pythonic, efficient code!